In [ ]:
from pprint import pprint
from dotenv import load_dotenv
from dataclasses import dataclass
import os

load_dotenv()

True

In [ ]:
# llm init
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    'gpt-5o-nano',
    api_key = os.getenv('OPENAI_API_KEY'),
)

In [ ]:
# 메모리 추가(멀티턴 대화 지원)
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

In [ ]:
# 대화 내용 유지할 쓰레드 설정
config = {"configurable": {"thread_id": "1"}}

In [72]:
# 웹 서치하는 tool, 응답 형식 정의
from langchain.tools import tool
import requests
from langchain.agents import create_agent

@tool
def web_search(query: str) -> str:
    """주어진 검색어로 웹에서 최신 정보를 찾아 요약합니다."""
    r = requests.get(f"https://api.duckduckgo.com/?q={query}&format=json")
    return r.json().get("AbstractText", "검색 결과 없음")

@dataclass
class ResponseFormat:
    """웹 검색 결과를 반환하는 구조화된 응답 형식입니다."""
    search_result: str
    source_url: str | None = None

In [73]:
# agent 생성
agent = create_agent(
    model='gpt-5-nano',
    tools=[web_search],
    response_format=ResponseFormat
)

In [74]:
@dataclass
class Context:
    """사용자별 런타임 정보를 담는 구조체 클래스입니다."""
    user_id: str
    preferred_currency = 'KRW'

In [75]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "최근 AI 기술 동향에 대해 알려줘."}]
    },
    config=config,
    context=Context(user_id="2"),
    tool_choice='any' # 도구 사용 강제(일반 llm으로의 fallback 방지)
)

pprint(response)

UnicodeEncodeError: 'ascii' codec can't encode character '\ud0a4' in position 185: ordinal not in range(128)

In [ ]:
# 서버 측 tool 사용
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    'gpt-5o-nano',
    api_key = os.getenv('OPENAI_API_KEY'),
)

tool = {"type": "web_search"} # llm 서버 측에서 제공하는 도구
model_with_tools = model.bind_tools([tool]) # langchain의 model 객체에 도구 바인딩

response = model_with_tools.invoke("오타니 월드시리즈 9출루 경기에 대해 설명해줘.")
pprint(response.content_blocks)

UnicodeEncodeError: 'ascii' codec can't encode character '\ud0a4' in position 185: ordinal not in range(128)

In [12]:
from langchain_core.callbacks.base import BaseCallbackHandler
import json
from pathlib import Path
import time

# 커스텀 트레이서 예시
class DetailedLocalTracer(BaseCallbackHandler):
    """
    LangChain Callback 기반의 세부 트레이싱 핸들러.
    각 단계별로 시작/종료 시각, latency, 입력/출력 크기 등을 콘솔 및 파일로 기록.
    """

    def __init__(self, log_path: str = "local_traces.jsonl"):
        self.log_file = Path(log_path)
        self.log_file.parent.mkdir(exist_ok=True, parents=True)
        self._events = []

    def _record(self, event_type: str, name: str, payload: dict):
        timestamp = time.time()
        record = {
            "ts": timestamp,
            "event": event_type,
            "name": name,
            "payload": payload,
        }
        self._events.append(record)
        # 콘솔 출력
        print(f"[TRACE] {event_type.upper():<10} | {name:<20} | {payload}")

    def _flush(self):
        # 파일에 JSONL 형식으로 저장
        with self.log_file.open("a", encoding="utf-8") as f:
            for ev in self._events:
                f.write(json.dumps(ev, ensure_ascii=False) + "\n")
        self._events.clear()

    # ──────────────────────────────────────────────
    # LLM Events
    # ──────────────────────────────────────────────
    def on_llm_start(self, serialized, prompts, **kwargs):
        self._record("llm_start", serialized.get("name", "LLM"), {"prompt_len": len(str(prompts))})

    def on_llm_end(self, response, **kwargs):
        generations = response.get("generations", [])
        output_text = generations[0][0].get("text", "") if generations else ""
        self._record("llm_end", "LLM", {"output_len": len(output_text)})
        self._flush()

    # ──────────────────────────────────────────────
    # Tool Events
    # ──────────────────────────────────────────────
    def on_tool_start(self, serialized, input_str, **kwargs):
        self._record("tool_start", serialized.get("name", "tool"), {"input_len": len(input_str)})

    def on_tool_end(self, output, **kwargs):
        self._record("tool_end", "tool", {"output_len": len(str(output))})
        self._flush()

    # ──────────────────────────────────────────────
    # Chain Events (전체 흐름)
    # ──────────────────────────────────────────────
    def on_chain_start(self, serialized, inputs, **kwargs):
        self._record("chain_start", serialized.get("name", "chain"), {"inputs": list(inputs.keys())})

    def on_chain_end(self, outputs, **kwargs):
        self._record("chain_end", "chain", {"outputs": list(outputs.keys())})
        self._flush()

    # ──────────────────────────────────────────────
    # Error Handling
    # ──────────────────────────────────────────────
    def on_chain_error(self, error, **kwargs):
        self._record("error", "chain", {"error": str(error)})
        self._flush()

In [13]:
# langchain tracing 사용
# langchain API key 필요(환경 변수로 관리)
from langchain_core.tracers.langchain import LangChainTracer


tracer = LangChainTracer(project_name="langchain_1.0_practice")
custom_tracer = DetailedLocalTracer(log_path="traces/local_trace.jsonl")

response = model.invoke(
    "재밌는 농담 목록 찾아줘",
    config={
        "run_name": "joke_search",
        "tags": ["humor", "demo"], 
        "metadata": {"user_id": "123"},
        "callbacks": [tracer, custom_tracer], # tracing용 로그 시스템. 커스텀 핸들러 / 기본 제공 핸들러 모두 사용 가능
    }
)

pprint(response)
print(tracer.get_run_url())  # 트레이스 URL 확인
print(custom_tracer.get_run_url())  # 그래프 뷰 URL 확인

[TRACE] LLM_START  | ChatOpenAI           | {'prompt_len': 24}


UnicodeEncodeError: 'ascii' codec can't encode character '\ud0a4' in position 185: ordinal not in range(128)

In [ ]:
# 멀티턴 대화 테스트
response = model.invoke(
    "뭔 소리야 농담이라니까",
    config={
        "run_name": "joke_search",
        "tags": ["humor", "demo"], 
        "metadata": {"user_id": "123"},
        "callbacks": [tracer], # tracing용 로그 시스템. 커스텀 핸들러 / 기본 제공 핸들러 모두 사용 가능
    }
)

pprint(response)
print(tracer.get_run_url())  # 트레이스 URL 확인

AIMessage(content='아, 농담이었군요! 제 대답이 너무 진지했나 봐요. 분위기를 맞춰서 간단한 농담 하나 드릴게요.\n\n- 왜 바나나가 길에서 넘어질까요? 껍질이 미끄러워서요.\n\n더 원하시면 이런 식으로 계속 가볍게 농담해 드릴게요. 특정 톤이나 주제가 있나요? 말장난, 짧은 이야기, 상황극 중에서 어떤 걸 원하시는지 알려주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2492, 'prompt_tokens': 15, 'total_tokens': 2507, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 2368, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CVrneclBdLFBc9WqVAq6OoMI2Gs2N', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--f00c660f-e43f-4bf7-9af8-f2a75a42b397-0', usage_metadata={'input_tokens': 15, 'output_tokens': 2492, 'total_tokens': 2507, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning'

In [ ]:
# 멀티턴 대화 테스트
response = model.invoke(
    "20대 청년들이 좋아할만한 걸로",
    config={
        "run_name": "joke_search",
        "tags": ["humor", "demo"], 
        "metadata": {"user_id": "123"},
        "callbacks": [tracer], # tracing용 로그 시스템. 커스텀 핸들러 / 기본 제공 핸들러 모두 사용 가능
    }
)

pprint(response)
print(tracer.get_run_url())  # 트레이스 URL 확인

AIMessage(content='좋아요! 의도에 따라 다르지만, 20대 청년들이 보통 좋아하는 아이템/아이디어를 몇 가지 범주로 뽑아봤어요. 필요에 맞게 골라보세요.\n\n- 기술/가전\n  - 무선 이어폰(노이즈 캔슬링)나 블루투스 스피커\n  - 스마트워치나 피트니스 트래커\n  - 게이밍 마우스/키보드나 USB-C 멀티허브\n  - 미니 포켓 빔프로젝터\n\n- 패션/스타일\n  - 미니멀한 티셔츠 2–3장 + 가디건 or 자켓\n  - 로우탑 스니커즈나 캡모자\n  - 실용적인 백팩이나 크로스백\n\n- 취미/라이프스타일\n  - 홈트용 덤벨 세트, 요가 매트\n  - 커피 애호가용 핸드드립 세트 + 원두 구독\n  - 보드게임이나 카드게임\n\n- 엔터테인먼트/콘텐츠\n  - 인기 게임 타이틀이나 게임 구독 서비스\n  - 그래픽 노블/만화, 음악 스트리밍 구독\n\n- 경험/체험\n  - 방탈출 카페나 VR 카페 이용권\n  - 근교 당일치기 여행 코스나 맛집 탐방 쿠폰\n\n- 자기계발/커리어\n  - 온라인 강의 구독(코딩, 디자인, 마케팅 등 관심 분야)\n  - 생산성 도구나 코워킹/협업 구독 서비스\n\n- 선물 아이디어\n  - 맞춤 인생 사진 포토북, 편지와 함께하는 소소한 선물 세트\n\n원하신다면 예산대별로 더 구체적인 추천(예: 5만원 이하, 5–15만원, 15만원 이상)이나 특정 관심사에 맞춘 리스트로 좁혀 드릴게요. 예산이나 대상(본인 취미를 위한 건지 선물용인지), 선호 분위기(심플/트렌디/고급) 같은 정보를 알려주실 수 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2542, 'prompt_tokens': 17, 'total_tokens': 2559, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0